# 齒輪分析主控制台

本專案用於分析兩個齒輪的重疊情況和振動特性，包含以下功能：
1. 齒輪模型載入和定位
2. 干涉分析
3. 3D 可視化顯示
4. FFT振動模擬

## 操作說明
- **X距離**: 控制小齒輪(Pinion)在X軸方向的位置
- **Y距離**: 控制大齒輪(Gear)在Y軸方向的位置

In [3]:
# 匯入所有必要的模組
import sys
import os

# 添加專案路徑
project_path = '/Users/rich/Documents/code/gear2D_proj/RL_project'
sys.path.append(project_path)

# 匯入自定義模組
from geometry.gear_loader import GearLoader
from geometry.gear_transformer import GearTransformer
from visualization.gear_visualizer import GearVisualizer
from analysis.gear_interference_analyzer import GearInterferenceAnalyzer
from simulation.gear_vibration_simulator import GearVibrationSimulator
from config_manager import ConfigManager
from analysis.vibration_data_analyzer import VibrationDataAnalyzer, get_vibration_analyzer, quick_load_latest

# 匯入其他必要的函式庫
import numpy as np
import plotly.graph_objects as go
from ipywidgets import interact, FloatSlider, IntSlider, Button, VBox, HBox, Output
import ipywidgets as widgets

print("所有模組載入完成！")
print("✅ 配置管理器已載入")
print("✅ 振動數據分析器已載入")

✅ 配置文件載入成功: /Users/rich/Documents/code/gear2D_proj/RL_project/config.json
所有模組載入完成！
✅ 配置管理器已載入
✅ 振動數據分析器已載入


In [4]:
# 初始化所有分析器
config = ConfigManager()
loader = GearLoader()
transformer = GearTransformer()
visualizer = GearVisualizer()
analyzer = GearInterferenceAnalyzer()
vibration_sim = GearVibrationSimulator()
vib_data_analyzer = VibrationDataAnalyzer()

print("分析器初始化完成！")
print("✅ 配置管理器初始化完成")
print("✅ 振動數據分析器初始化完成")

# 載入齒輪模型
print("\n載入齒輪模型...")
pinion_mesh, gear_mesh = loader.load_stl_files()

if pinion_mesh is not None and gear_mesh is not None:
    transformer.setup_gears(pinion_mesh, gear_mesh)
    print("齒輪模型設置完成！")
else:
    print("錯誤：無法載入齒輪模型")

✅ 配置文件載入成功: /Users/rich/Documents/code/gear2D_proj/RL_project/config.json
✅ 配置文件載入成功: /Users/rich/Documents/code/gear2D_proj/RL_project/config.json
干涉分析器初始化完成
  輕微干涉閾值: 0.5 mm (重疊深度)
  中度干涉閾值: 1.0 mm (重疊深度)
  嚴重干涉閾值: 2.0 mm (重疊深度)
  接觸閾值: 0.5 mm (間隙)
  接近接觸閾值: 1.0 mm (間隙)
✅ 配置文件載入成功: /Users/rich/Documents/code/gear2D_proj/RL_project/config.json
分析器初始化完成！
✅ 配置管理器初始化完成
✅ 振動數據分析器初始化完成

載入齒輪模型...
成功載入齒輪檔案:
- 小齒輪: /Users/rich/Documents/code/gear2D_proj/RL_project/STL_data/Pinion_1TH00038_v2.0.STL
- 大齒輪: /Users/rich/Documents/code/gear2D_proj/RL_project/STL_data/Gear_1TH00037_v2.0.STL
齒輪模型設置完成！


## 齒輪位置控制面板

使用下方的滑桿來調整齒輪位置：

In [ ]:
# 建立互動式控制面板
class GearControlPanel:
    def __init__(self, transformer, visualizer, analyzer, vibration_sim, vib_data_analyzer):
        self.transformer = transformer
        self.visualizer = visualizer
        self.analyzer = analyzer
        self.vibration_sim = vibration_sim
        self.vib_data_analyzer = vib_data_analyzer
        
        # 建立滑桿 (擴大範圍以測試極端碰撞)
        self.x_slider = FloatSlider(
            value=24, min=-10, max=50, step=1,
            description='X距離 (Pinion):',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        self.y_slider = FloatSlider(
            value=-31, min=-80, max=20, step=1,
            description='Y距離 (Gear):',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        self.offset_slider = FloatSlider(
            value=10.0, min=-45, max=45, step=1,
            description='旋轉偏移 (度):',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        self.sample_rate_slider = IntSlider(
            value=100, min=20, max=120, step=1,
            description='分析取樣率:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='400px')
        )
        
        # 建立按鈕
        self.analyze_button = Button(
            description='🔍 執行分析',
            button_style='primary',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        self.vibration_button = Button(
            description='📊 振動分析',
            button_style='success',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        self.reset_button = Button(
            description='🔄 重置',
            button_style='warning',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        # 新增快速測試按鈕
        self.collision_test_button = Button(
            description='💥 碰撞測試',
            button_style='danger',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        # 新增梯度測試按鈕
        self.gradient_test_button = Button(
            description='📈 梯度測試',
            button_style='info',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        # 新增數據管理按鈕
        self.list_data_button = Button(
            description='📁 查看數據',
            button_style='',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        self.compare_data_button = Button(
            description='📊 比較數據',
            button_style='',
            layout=widgets.Layout(width='150px', height='40px')
        )
        
        # 輸出區域
        self.output = Output()
        
        # 設置事件處理
        self.analyze_button.on_click(self.on_analyze_click)
        self.vibration_button.on_click(self.on_vibration_click)
        self.reset_button.on_click(self.on_reset_click)
        self.collision_test_button.on_click(self.on_collision_test_click)
        self.gradient_test_button.on_click(self.on_gradient_test_click)
        self.list_data_button.on_click(self.on_list_data_click)
        self.compare_data_button.on_click(self.on_compare_data_click)
        
        # 儲存當前分析結果
        self.current_analysis = None
        
    def create_panel(self):
        """建立控制面板"""
        sliders_box = VBox([
            self.x_slider,
            self.y_slider,
            self.offset_slider,
            self.sample_rate_slider
        ])
        
        buttons_box1 = HBox([
            self.analyze_button,
            self.vibration_button,
            self.reset_button
        ])
        
        buttons_box2 = HBox([
            self.collision_test_button,
            self.gradient_test_button
        ])
        
        buttons_box3 = HBox([
            self.list_data_button,
            self.compare_data_button
        ])
        
        return VBox([sliders_box, buttons_box1, buttons_box2, buttons_box3, self.output])
    
    def on_list_data_click(self, button):
        """列出儲存的振動數據"""
        with self.output:
            self.output.clear_output()
            print("📁 查看儲存的振動數據...")
            files = self.vib_data_analyzer.list_saved_files()
            
            if files:
                print(f"\n💡 提示：使用以下指令分析特定文件：")
                print(f"   vib_data_analyzer.analyze_single_file('/Users/rich/Documents/code/gear2D_proj/tmp/文件名.npz')")
                print(f"   或使用 quick_load_latest() 快速載入最新文件")
            else:
                print("📂 tmp資料夾中沒有振動數據文件")
                print("💡 執行振動分析後會自動儲存數據到tmp資料夾")
    
    def on_compare_data_click(self, button):
        """比較最近的振動數據，專注於GMF能量、旁波和諧波分析"""
        with self.output:
            self.output.clear_output()
            print("📊 開始進行增強版振動數據比較分析...")
            print("🔧 重點分析項目：")
            print("   • GMF(齒輪嚙合頻率)基頻能量")
            print("   • GMF諧波能量分布 (2×GMF, 3×GMF, ...)")
            print("   • GMF旁波分析 (調變頻率檢測)")
            print("   • 高頻諧波能量分布")
            print("   • 齒輪故障特徵頻率分析")
            
            files = self.vib_data_analyzer.list_saved_files()
            if len(files) < 2:
                print("\n⚠️ 需要至少2個振動數據文件進行比較")
                print("💡 請執行更多的振動分析以產生比較數據")
                return
            
            # 選擇最近的5個文件進行比較
            recent_files = files[:5]
            filepaths = [os.path.join(self.vib_data_analyzer.tmp_path, f) for f in recent_files]
            
            print(f"\n🔍 比較最近的 {len(recent_files)} 個振動數據文件...")
            print("   分析中，請稍候...")
            
            self.vib_data_analyzer.compare_vibration_data(filepaths)
            
            print(f"\n✅ 增強版振動數據比較分析完成！")
            print(f"📈 主要關注指標：")
            print(f"   • GMF能量等級 - 反映嚙合品質")
            print(f"   • 諧波比例 - 檢測齒面磨損")
            print(f"   • 旁波強度 - 診斷調變和不對中")
            print(f"   • 高頻能量 - 監控軸承和精密度")
    
    def on_gradient_test_click(self, button):
        """執行梯度測試 - 測試從正常到碰撞的完整梯度"""
        with self.output:
            self.output.clear_output()
            print("📈 開始執行干涉梯度測試...")
            
            # 測試更廣泛的梯度場景，包含Y位置變化
            test_scenarios = [
                {"name": "大間隙", "x": 35, "y": -31, "offset": 10},
                {"name": "正常間隙", "x": 30, "y": -31, "offset": 10},
                {"name": "小間隙", "x": 26, "y": -31, "offset": 10},
                {"name": "接觸邊緣", "x": 24, "y": -31, "offset": 10},
                {"name": "輕微干涉", "x": 22, "y": -31, "offset": 10},
                {"name": "中度干涉", "x": 20, "y": -31, "offset": 10},
                {"name": "嚴重干涉", "x": 18, "y": -31, "offset": 10},
                {"name": "Y軸偏移", "x": 20, "y": -28, "offset": 10},
                {"name": "極端碰撞", "x": 15, "y": -28, "offset": 15},
            ]
            
            results = []
            last_successful_analysis = None  # 儲存最後一次成功的分析結果
            
            for scenario in test_scenarios:
                try:
                    print(f"\n🔍 測試場景: {scenario['name']}")
                    print(f"   設定: X={scenario['x']}, Y={scenario['y']}, 偏移={scenario['offset']}°")
                    
                    # 重置齒輪到原始狀態
                    transformer.reset_gears()
                    
                    # 執行變換
                    vp, fp, vg, fg, transform_info = transformer.transform_gears(
                        x_distance=scenario['x'],
                        y_distance=scenario['y'],
                        manual_offset_deg=scenario['offset']
                    )
                    
                    # 執行干涉分析 (修正調用方式)
                    analysis = analyzer.analyze_interference(vp, fp, vg, fg, sample_rate=self.sample_rate_slider.value)
                    last_successful_analysis = analysis  # 儲存成功的分析結果
                    
                    # 顯示結果
                    if 'severity' in analysis:
                        severity = analysis['severity']
                        center_dist = transform_info.get('center_distance', 0)
                        print(f"   中心距離: {center_dist:.2f} mm")
                        print(f"   結果: {severity['severity_level']} ({severity['severity_score']:.1f}/100)")
                        print(f"   干涉點: {analysis['statistics']['total_interference_points']}")
                        
                        results.append({
                            'scenario': scenario['name'],
                            'x': scenario['x'],
                            'y': scenario['y'],  # 新增Y位置
                            'center_distance': center_dist,
                            'score': severity['severity_score'],
                            'level': severity['severity_level'],
                            'points': analysis['statistics']['total_interference_points']
                        })
                    else:
                        print("   ⚠️ 無法獲取嚴重程度資訊")
                        
                except Exception as e:
                    print(f"   ❌ 測試失敗: {e}")
                    continue  # 繼續下一個測試
            
            # 顯示詳細比較結果 (新增Y位置欄)
            if results:
                print(f"\n📊 完整梯度測試結果:")
                print(f"{'場景':<12} {'X位置':<6} {'Y位置':<6} {'中心距':<8} {'嚴重程度':<10} {'分數':<8} {'干涉點':<8}")
                print("-" * 78)
                for result in results:
                    print(f"{result['scenario']:<12} {result['x']:<6} {result['y']:<6} {result['center_distance']:<8.1f} "
                          f"{result['level']:<10} {result['score']:<8.1f} {result['points']:<8}")
                
                # 分析梯度品質
                if len(results) >= 3:
                    scores = [r['score'] for r in results]
                    print(f"\n📈 梯度分析:")
                    print(f"   分數範圍: {min(scores):.1f} ~ {max(scores):.1f}")
                    print(f"   最大差值: {max(scores) - min(scores):.1f}")
                    
                    # 檢查是否有合理的梯度
                    if max(scores) - min(scores) > 30:
                        print("   ✅ 梯度範圍良好")
                    else:
                        print("   ⚠️ 梯度範圍可能過小")
                        
                    # 檢查是否有正常範圍
                    normal_count = sum(1 for s in scores if s < 30)
                    if normal_count > 0:
                        print(f"   ✅ 發現 {normal_count} 個正常/輕微干涉場景")
                    else:
                        print("   ⚠️ 沒有發現正常間隙場景")
            else:
                print("❌ 沒有成功的測試結果")
            
            # 更新最後一個成功測試的可視化
            if last_successful_analysis is not None:
                self.current_analysis = last_successful_analysis
                print(f"\n✅ 梯度測試完成！最後成功場景的分析結果已載入。")
            else:
                print(f"\n⚠️ 梯度測試完成，但沒有成功的分析結果。")
    
    def on_collision_test_click(self, button):
        """執行極端碰撞測試"""
        with self.output:
            self.output.clear_output()
            print("💥 開始執行極端碰撞測試...")
            
            # 測試多個極端碰撞場景
            test_scenarios = [
                {"name": "正常間隙", "x": 28, "y": -31, "offset": 10},
                {"name": "輕微接近", "x": 24, "y": -31, "offset": 10},
                {"name": "中度干涉", "x": 20, "y": -31, "offset": 10},
                {"name": "嚴重碰撞", "x": 15, "y": -31, "offset": 10},
                {"name": "極端碰撞", "x": 10, "y": -25, "offset": 20},
            ]
            
            results = []
            
            for scenario in test_scenarios:
                try:
                    print(f"\n🔍 測試場景: {scenario['name']}")
                    print(f"   設定: X={scenario['x']}, Y={scenario['y']}, 偏移={scenario['offset']}°")
                    
                    # 重置齒輪到原始狀態
                    transformer.reset_gears()
                    
                    # 執行變換
                    vp, fp, vg, fg, transform_info = transformer.transform_gears(
                        x_distance=scenario['x'],
                        y_distance=scenario['y'],
                        manual_offset_deg=scenario['offset']
                    )
                    
                    # 執行干涉分析 (修正調用方式)
                    analysis = analyzer.analyze_interference(vp, fp, vg, fg, sample_rate=self.sample_rate_slider.value)
                    
                    # 顯示結果
                    if 'severity' in analysis:
                        severity = analysis['severity']
                        print(f"   結果: {severity['severity_level']} ({severity['severity_score']:.1f}/100)")
                        print(f"   描述: {severity['description']}")
                        print(f"   干涉點: {analysis['statistics']['total_interference_points']}")
                        
                        results.append({
                            'scenario': scenario['name'],
                            'score': severity['severity_score'],
                            'level': severity['severity_level'],
                            'points': analysis['statistics']['total_interference_points']
                        })
                    else:
                        print("   ⚠️ 無法獲取嚴重程度資訊")
                        
                except Exception as e:
                    print(f"   ❌ 測試失敗: {e}")
            
            # 顯示比較結果
            if results:
                print(f"\n📊 碰撞測試結果比較:")
                print(f"{'場景':<12} {'嚴重程度':<10} {'分數':<8} {'干涉點':<8}")
                print("-" * 45)
                for result in results:
                    print(f"{result['scenario']:<12} {result['level']:<10} {result['score']:<8.1f} {result['points']:<8}")
                
                # 更新最後一個測試的可視化
                self.current_analysis = analysis
                print(f"\n✅ 碰撞測試完成！最後場景的分析結果已載入。")
    
    def on_analyze_click(self, button):
        """執行齒輪分析"""
        with self.output:
            self.output.clear_output()
            print("🔍 開始執行齒輪分析...")
            
            try:
                # 重置齒輪到原始狀態
                transformer.reset_gears()
                
                # 執行變換
                vp, fp, vg, fg, transform_info = transformer.transform_gears(
                    x_distance=self.x_slider.value,
                    y_distance=self.y_slider.value,
                    manual_offset_deg=self.offset_slider.value
                )
                
                # 創建基本可視化
                fig = visualizer.create_basic_visualization(vp, fp, vg, fg, transform_info)
                fig.show()
                
                # 執行干涉分析 (修正調用方式)
                print("\n📊 執行干涉分析...")
                self.current_analysis = analyzer.analyze_interference(vp, fp, vg, fg, sample_rate=self.sample_rate_slider.value)
                
                # 添加干涉可視化
                if 'interference_points' in self.current_analysis:
                    visualizer.add_interference_visualization(
                        self.current_analysis['interference_points']
                    )
                    visualizer.fig.show()
                
                print("\n✅ 分析完成！")
                
            except Exception as e:
                print(f"❌ 分析過程中發生錯誤: {e}")
                import traceback
                traceback.print_exc()
    
    def on_vibration_click(self, button):
        """執行振動分析"""
        with self.output:
            if self.current_analysis is None:
                print("⚠️ 請先執行齒輪分析！")
                return
            
            print("\n📊 開始執行振動分析...")
            
            try:
                # 模擬振動信號 (使用配置文件中的正確RPM)
                vibration_data = vibration_sim.simulate_vibration_signal(self.current_analysis)
                
                # 創建振動分析圖表
                vibration_fig = vibration_sim.plot_vibration_analysis(vibration_data)
                if vibration_fig:
                    vibration_fig.show()
                
                # 儲存振動資料到tmp資料夾
                filename_prefix = f"vibration_x{self.x_slider.value}_y{self.y_slider.value}"
                saved_path = self.vib_data_analyzer.save_vibration_data(vibration_data, filename_prefix)
                
                # 顯示分析摘要
                vibration_sim.print_analysis_summary(vibration_data)
                
                print(f"\n💾 振動數據已自動儲存到tmp資料夾")
                print(f"📁 文件路徑: {saved_path}")
                print("\n✅ 振動分析完成！")
                
            except Exception as e:
                print(f"❌ 振動分析過程中發生錯誤: {e}")
                import traceback
                traceback.print_exc()
    
    def on_reset_click(self, button):
        """重置參數"""
        with self.output:
            self.output.clear_output()
            self.x_slider.value = 24
            self.y_slider.value = -31
            self.offset_slider.value = 10.0
            self.sample_rate_slider.value = 100
            self.current_analysis = None
            print("🔄 參數已重置")

# 建立控制面板
control_panel = GearControlPanel(transformer, visualizer, analyzer, vibration_sim, vib_data_analyzer)
panel_widget = control_panel.create_panel()

display(panel_widget)